<a href="https://colab.research.google.com/github/dqniellew1/DLPT/blob/master/dlpt_tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
a = [1.0, 2.0, 1.0]

In [0]:
a[0]

1.0

In [0]:
import torch

Constructing first tensor

In [0]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [0]:
a[1]

tensor(1.)

In [0]:
a[2] = 2.0

In [0]:
a

tensor([1., 1., 2.])

Sample: Coordinate points

In [0]:
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0


In [0]:
points

tensor([4., 1., 5., 3., 2., 1.])

Pass a python list into a tensor

In [0]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0 ,2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

Get coordinates of the first point

In [0]:
float(points[0]), float(points[1])

(4.0, 1.0)

2D tensor

In [0]:
points = torch.tensor([[4.0, 2.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 2.],
        [5., 3.],
        [2., 1.]])

In [0]:
points.shape

torch.Size([3, 2])

Initialize tensor with zeros and ones

In [0]:
points = torch.zeros(3,2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [0]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [0]:
points[0,1]

tensor(1.)

In [0]:
points[0]

tensor([4., 1.])

## Indexing tensors

In [0]:
some_list = list(range(6))

In [0]:
some_list[:] # returns all elements of the list
some_list[1:4] # returns elements at position 1 to 3 excluding 4
some_list[1:] # returns all elements after position 1
some_list[:4] # returns all elements up to 4 
some_list[:-1] # returns all elements from the start to one before the last element
some_list[1:4:2] # returns elements from 1 to 3 with steps of 2

[1, 3]

In [0]:
points[1:] # 1st row onwards
points[1:, :] # 1st row onwards and all columns
points[1:, 0] # 1st row onwards and 0th column
points[None] # Add dimension of size one, just like unsqueeze

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

Imagine we have a 3D tensor, we want to convert it to grayscale.

In [0]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722]) # typical weights for colors to derive a single brightnes value

Pretend we have an additional dimension; of batch size

In [0]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

Sometimes RGB channels are in dimension 0 or 1. We can generalize by counting from the end: That is, they are always in dimension -3. 

In [0]:
img_gray_naive = img_t.mean(-3) # get mean of channels
batch_gray_naive = batch_t.mean(-3) # get mean of channels
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [0]:
# Can only multiply things that are of the same shape
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1) # Add 2 dimensions
img_weights = (img_t * unsqueezed_weights) # Multiply by same D
batch_weights = (batch_t * unsqueezed_weights) # broadcasting appends leading dimensions of size one automatically.
img_gray_weights = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

Above operations gets messy quickly, therefore the PyTorch function `einsum` makes it easier.

In [0]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

There is quite some bookkeeping invloved when doing tensor operations, therefore practitioners suggest giving names to the dimensions instead.

Enter named tensors

In [0]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels']);
weights_named

/pytorch/c10/core/TensorImpl.h:806: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable.


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

When we already have tensors and only want to add names, we can call the method `refine_names`. the ellipsis `...` allows you to leave out any number of dimensions. With the `rename` sibling you can also overwrite or drop (by passing in `None`) existing names.

In [0]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print(f'img named: {img_named.shape}, {img_named.names}')
print(f'batch named: {batch_named.shape}, {batch_named.names}')

img named: torch.Size([3, 5, 5]), ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]), (None, 'channels', 'rows', 'columns')


In addition to the dimension checks, eg same size, or broadcasted, PyTorch will check names for us. It does not automatically align dimensions, so we need to do this explicitly. `align_as` returns a tensor with missing dimensions added and existing ones permuted to the right order.

In [0]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

Functions accepting dimension arguments eg: `sum`, `mean` also take named dimensions

In [0]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

Combining dimensions with different names give an **error**!

In [0]:
gray_named= (img_named[..., :3] * weights_named).sum('channels')


RuntimeError: ignored

If we want to use tensors outside functions to operate on named tensors, we have to drop names by renaming them with `None`.

In [0]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

In [0]:
points > 1.0

tensor([[ True, False],
        [ True,  True],
        [ True, False]])

## Attributing `dtypes` to a tensor

In [0]:
double_points = torch.ones(10,2, dtype=torch.double)
short_points = torch.tensor([[1,2], [3,4]], dtype=torch.short)

In [0]:
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

Cast to assignt `dtype`

In [0]:
double_points = torch.zeros(10,2).double()
short_points = torch.ones(10, 2).short()

`to` method

In [0]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(torch.short)

When mixing input types in operations, inputs are automatically converted to larger type. If we want 32 bit, make sure all inputs are 32 bits.

In [0]:
points_64 = torch.rand(5, dtype=torch.double)
points_short = points_64.to(torch.short)
points_64 * points_short 

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

## Torch operations

`tranpose`

In [0]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [0]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

Tensors operation in PyTorch are divided into groups
- Creations ops like `ones` and `from_numpy`
- Mutating ops like `transpose` and changing the shape and content of tensors
- Maths ops
  * Pointwise ops: `abs`, `cos`
  * Reduction ops: `mean`, `std`, `norm`
  * Comparison ops: `equal`, `max`
  * Spectral ops: `stft`, `hamming_window`
  * Other ops: `cross`, `trace`
- Random sampling like `randn`, `normal`
- Serialization like `load`, `save`
- Parellelism like `set_num_threads`

Indexing into storage

In [0]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [0]:
points_storage = points.storage()
points_storage[0]

4.0

In [0]:
points.storage()[1]

1.0

In [0]:
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

Modifying Stored Values - Inplace operations

In [0]:
a = torch.ones(3, 2)

a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

Views over another tensor's storage

In [0]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [0]:
second_point.size()

torch.Size([2])

In [0]:
second_point.shape

torch.Size([2])

In [0]:
points.stride()

(2, 1)

In [0]:
second_point.storage_offset()

2

In [0]:
second_point.stride()

(1,)

Changing a sub-tensor will have a side-effect on the orginal tensor as they share the same memory allocation

In [0]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [0]:
second_point

tensor([5., 3.])

In [0]:
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

This might not be desirable so we can clone the sub-tensor into a new tensor

In [0]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

Tranposing without copying

In [0]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [0]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [0]:
id(points.storage()) == id(points_t.storage())

True

They have the same id but only differ in stride

In [0]:
points.stride()

(2, 1)

In [0]:
points_t.stride()

(1, 2)

Transposing in higher dimensions

In [0]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)

In [0]:
some_t.shape

torch.Size([3, 4, 5])

In [0]:
transpose_t.shape

torch.Size([5, 4, 3])

In [0]:
some_t.stride()

(20, 5, 1)

In [0]:
transpose_t.stride()

(1, 5, 20)

Contiguous tensors

In [0]:
points.is_contiguous()

True

In [0]:
points_t.is_contiguous()

False

In [0]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [0]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [0]:
points_t.stride()

(1, 2)

In [0]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [0]:
points_t_cont.stride()

(3, 1)

In [0]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]

Convert tensor to numpy array

In [0]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

Vice-versa

In [0]:
points = torch.from_numpy(points_np)

Managing a tnesor's device atribute

In [0]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])

In [0]:
points_gpu = points.to(device='cuda')

From here on out, any operations on a tensor is carried out on the GPU

In [0]:
points = 2 * points

In [0]:
points_gpu = 2 * points.to(device='cuda')

In [0]:
points_gpu = points_gpu + 4

Serializing or saving tensors

In [0]:
torch.save(points, 'ourpoints.t')

In [0]:
with open('ourpoints.t', 'wb') as f:
  torch.save(points, f)

Loading our points back

In [0]:
points = torch.load('ourpoints.t')

In [0]:
with open('ourpoints.t', 'rb') as f:
  points = torch.load(f)

In [0]:
import torch

Exercise

In [0]:
a = torch.tensor(list(range(9)))
a

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [0]:
a.shape, a.stride(), a.storage_offset()

(torch.Size([9]), (1,), 0)

In [0]:
b = a.view(3, 3)
b

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

View reshapes the tensor, in this case into a 3 by 3 shape, 2D tensor

In [0]:
id(a.storage()) == id(b.storage()) # They share the same storage

True

In [0]:
c = b[1:, 1:] # 2x2
c

tensor([[4, 5],
        [7, 8]])

In [0]:
c.shape, c.stride(), c.storage_offset()

(torch.Size([2, 2]), (3, 1), 4)

In [0]:
a.to(torch.double)

In [0]:
torch.cos(a)

RuntimeError: ignored